In [15]:
import json
from pprint import pprint
import re
from pyspark.sql import SparkSession
import nltk.data

spark  = SparkSession.builder.master('local').appName('sou').getOrCreate()
sc = spark.sparkContext
lotr = spark.sparkContext.wholeTextFiles('/scratch/midway2/ceidsmoe').values()

In [3]:
tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')
sentences = lotr.flatMap(lambda x: tokenizer.tokenize(x))
sentences.count()

31351

In [14]:
import nltk
import inflect
from operator import add

arpabet = nltk.corpus.cmudict.dict()
iengine = inflect.engine()

def is_number(s):
    try:
        f = int(s)
        return True
    except:
        return False

alpha = 0.05
    
def euphony_measure(sentence):
    phonemes = []
    exceptions = []
    for word in re.split('\W+', sentence):
        if word == '':
            continue
        word = word.lower()
        if is_number(word):
            word = iengine.number_to_words(int(word))
        try:
             phonemes.append(arpabet[word][0])
        except Exception as e:
            exceptions.append((word, 1))
            
    return exceptions

phonemes = sc.parallelize(sentences.map(euphony_measure).reduce(add)).reduceByKey(add)
phonemes.count()

4407